<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [24]:
import numpy as np
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [4]:
#ANSWER
response.json()

{'timestamp': 1682113597,
 'iss_position': {'latitude': '4.5712', 'longitude': '74.7160'},
 'message': 'success'}

In [6]:
#ANSWER


We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [7]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [8]:
#ANSWER
header=response.headers
print(header)

{'Date': 'Sat, 22 Apr 2023 02:52:53 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}


Print the content of the response (the data that the server returned):

In [10]:
#ANSWER
content= response.content
print(content)

b'{"api_status":"ALPHA","request_timestamp":"2023-04-22T02:52:53.275393051Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2023-04-21T20:50:51.542342724Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":14,"passes":[{"start":"2023-04-22T04:47:28.261Z","tca":"2023-04-22T04:50:58.261Z","end":"2023-04-22T04:54:58.261Z","aos_azimuth":12,"los_azimuth":100,"max_elevation":6.0},{"start":"2023-04-22T06:22:03.261Z","tca":"2023-04-22T06:27:33.261Z","end":"2023-04-22T06:32:58.261Z","aos_azimuth":313,"los_azimuth":133,"max_elevation":79.0},{"start":"2023-04-22T08:00:03.261Z","tca":"2023-04-22T08:04:33.261Z","end":"2023-04-22T08:09:13.261Z","aos_azimuth":263,"los_azimuth":149,"max_elevation":12.0},{"start":"2023-04-22T09:39:53.261Z","tca":"2023-04-22T09:42:53.261Z","end":"2023-04-22T09:45:48.261Z","aos_azimuth":219,"los_azimuth":153,"max_elevation":3.0},{"start":"2023-04-22T11:17:48.261Z","tca":"2023-04-22T11:21:18.261Z","end":"2023-04-22T11:24:48.261Z","aos_azimut

Note that this is a Python byte string:

In [11]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [12]:
#ANSWER
content_type= response.headers['content-type']
print(content_type)

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [13]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2023-04-22T02:52:53.275393051Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2023-04-21T20:50:51.542342724Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 14, 'passes': [{'start': '2023-04-22T04:47:28.261Z', 'tca': '2023-04-22T04:50:58.261Z', 'end': '2023-04-22T04:54:58.261Z', 'aos_azimuth': 12, 'los_azimuth': 100, 'max_elevation': 6.0}, {'start': '2023-04-22T06:22:03.261Z', 'tca': '2023-04-22T06:27:33.261Z', 'end': '2023-04-22T06:32:58.261Z', 'aos_azimuth': 313, 'los_azimuth': 133, 'max_elevation': 79.0}, {'start': '2023-04-22T08:00:03.261Z', 'tca': '2023-04-22T08:04:33.261Z', 'end': '2023-04-22T08:09:13.261Z', 'aos_azimuth': 263, 'los_azimuth': 149, 'max_elevation': 12.0}, {'start': '2023-04-22T09:39:53.261Z', 'tca': '2023-04-22T09:42:53.261Z', 'end': '2023-04-22T09:45:48.261Z', 'aos_azimuth': 219, 'los_azimuth': 153, 'max_elevation': 3.0}, {'start': '2023-04-22T11:17:48.261Z', 'tca': '

What kind of object did this give us?

#ANSWER:
**This gave us the dictionaries**

Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the passes value to a separate list called passes' instead :

In [22]:
#ANSWER:
passes= overheads['passes']
print(passes)

[{'start': '2023-04-22T04:47:28.261Z', 'tca': '2023-04-22T04:50:58.261Z', 'end': '2023-04-22T04:54:58.261Z', 'aos_azimuth': 12, 'los_azimuth': 100, 'max_elevation': 6.0}, {'start': '2023-04-22T06:22:03.261Z', 'tca': '2023-04-22T06:27:33.261Z', 'end': '2023-04-22T06:32:58.261Z', 'aos_azimuth': 313, 'los_azimuth': 133, 'max_elevation': 79.0}, {'start': '2023-04-22T08:00:03.261Z', 'tca': '2023-04-22T08:04:33.261Z', 'end': '2023-04-22T08:09:13.261Z', 'aos_azimuth': 263, 'los_azimuth': 149, 'max_elevation': 12.0}, {'start': '2023-04-22T09:39:53.261Z', 'tca': '2023-04-22T09:42:53.261Z', 'end': '2023-04-22T09:45:48.261Z', 'aos_azimuth': 219, 'los_azimuth': 153, 'max_elevation': 3.0}, {'start': '2023-04-22T11:17:48.261Z', 'tca': '2023-04-22T11:21:18.261Z', 'end': '2023-04-22T11:24:48.261Z', 'aos_azimuth': 204, 'los_azimuth': 125, 'max_elevation': 5.0}, {'start': '2023-04-22T12:54:03.261Z', 'tca': '2023-04-22T12:59:03.261Z', 'end': '2023-04-22T13:04:18.261Z', 'aos_azimuth': 214, 'los_azimuth': 

Now extract the `start` strings into an array called `srisetimes`:

In [26]:
#ANSWER:
srisetimes=[x['start'] for x in passes]
print(srisetimes)

['2023-04-22T04:47:28.261Z', '2023-04-22T06:22:03.261Z', '2023-04-22T08:00:03.261Z', '2023-04-22T09:39:53.261Z', '2023-04-22T11:17:48.261Z', '2023-04-22T12:54:03.261Z', '2023-04-22T14:30:38.261Z', '2023-04-23T04:01:08.261Z']


These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [32]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]


Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [33]:
#ANSWER:
for rt in risetimes:
    print(rt.strftime('%d/%m/%y %I:%M %p'))

22/04/23 04:47 AM
22/04/23 06:22 AM
22/04/23 08:00 AM
22/04/23 09:39 AM
22/04/23 11:17 AM
22/04/23 12:54 PM
22/04/23 02:30 PM
23/04/23 04:01 AM


Finally, here is an endpoint that tells us who is on board:

In [30]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [45]:
#ANSWER:
data=response.json()
number_of_austronauts= data['number']
austronaut_name=[austronaut['name'] for austronaut in data['people']]
print(f"Number of astronauts: {number_of_austronauts}")
print("Astronaut_name")


Number of astronauts: 10
Astronaut_name


>

>

>



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



